# Data Preparation

## Libraries

In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Activation, Dense

Using TensorFlow backend.


## Directories

In [2]:
pwd

'/Users/mina/2_Bootcamp/cnn/drone_images'

In [3]:
test_dir='./test'
val_dir='./val'
train_dir='/Users/mina/2_Bootcamp/cnn/drone_images/train'

In [4]:
train_dir_one = train_dir + '/one'
train_dir_zero = train_dir + '/zero'

val_dir_one = val_dir + '/one'
val_dir_zero = val_dir + '/zero'

test_dir = test_dir
# test_dir_one = test_dir + '/one'
# test_dir_zero = test_dir + '/zero'

In [5]:
test = os.listdir(test_dir)
test.remove('.DS_Store')

In [6]:
print('number of Class 0 images in training images - ',len(os.listdir(train_dir_zero)))
print('number of Class 1 images in training images - ',len(os.listdir(train_dir_one)))

print('number of Class 0 images in testing images - ',len(os.listdir(val_dir_zero)))
print('number of Class 1 images in testing images - ',len(os.listdir(val_dir_one)))

print('number of images in testing images - ',len(os.listdir(test_dir)))

# print('number of 1 images in testing images - ',len(os.listdir(test_dir_one)))
# print('number of 0 images in testing images - ',len(os.listdir(test_dir_zero)))

number of Class 0 images in training images -  2954
number of Class 1 images in training images -  2417
number of Class 0 images in testing images -  1181
number of Class 1 images in testing images -  966
number of images in testing images -  3226


# Model with 3 blocks

## With 'relu'

In [7]:
# create data generator
datagen = ImageDataGenerator(rescale=1.0/255.0)
    
# prepare iterators
train_it = datagen.flow_from_directory(directory = train_dir,
                                        target_size=(200, 200),
                                        batch_size=64,
                                        class_mode='binary')
    
test_it = datagen.flow_from_directory(directory = val_dir,
                                      target_size=(200, 200),
                                      batch_size=64,
                                      class_mode='binary')

Found 5371 images belonging to 2 classes.
Found 2147 images belonging to 2 classes.


In [10]:
model = Sequential()

# Step 1 - Convolution
model.add(Conv2D(32, (3, 3), activation='relu', input_shape = (200, 200, 3)))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [11]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 198, 198, 32)      896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 99, 99, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 97, 97, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 46, 46, 128)       73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 23, 23, 128)       0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 67712)            

In [ ]:
batch_size = 64
history = model.fit_generator(train_it, 
                              steps_per_epoch=np.ceil(train_it.samples / batch_size),
                              validation_data=test_it,
                              validation_steps=np.ceil(test_it.samples / batch_size), 
                              epochs=10)
# evaluate model
_, acc = model.evaluate_generator(test_it, steps=len(test_it))
print('> %.3f' % (acc * 100.0))

In [ ]:
plt.style.use('bmh')
fig = plt.figure(figsize = (10,8))

plt.subplot(211)
plt.title('Cross Entropy Loss')
plt.plot(history.history['loss'], color='cyan', label='train')
plt.plot(history.history['val_loss'], color='magenta', label='validation')
plt.legend(loc='upper right')
    # plot accuracy
plt.subplot(212)
plt.title('Classification Accuracy')
plt.plot(history.history['accuracy'], color='cyan', label='train')
plt.plot(history.history['val_accuracy'], color='magenta', label='validation')
plt.legend(loc='upper right')
    # save plot to file
filename = sys.argv[0].split('/')[-1]
plt.savefig(filename + '_plot.png')
plt.show()

## With 'selu'

In [ ]:
# create data generator
datagen = ImageDataGenerator(rescale=1.0/255.0)
    
# prepare iterators
train_it = datagen.flow_from_directory(directory = train_dir,
                                        target_size=(200, 200),
                                        batch_size=64,
                                        class_mode='binary')
    
test_it = datagen.flow_from_directory(directory = val_dir,
                                      target_size=(200, 200),
                                      batch_size=64,
                                      class_mode='binary')

In [ ]:
model = Sequential()

# Step 1 - Convolution
model.add(Conv2D(32, (3, 3), activation='selu', padding='same', input_shape = (200, 200, 3)))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3, 3), activation='selu', padding='same'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(128, (3, 3), activation='selu', padding='same'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='selu'))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
batch_size = 64
history = model.fit_generator(train_it, 
                              steps_per_epoch=np.ceil(train_it.samples / batch_size),
                              validation_data=test_it,
                              validation_steps=np.ceil(test_it.samples / batch_size), 
                              epochs=20)
# evaluate model
_, acc = model.evaluate_generator(test_it, steps=len(test_it))
print('> %.3f' % (acc * 100.0))

In [ ]:
plt.style.use('bmh')
fig = plt.figure(figsize = (10,8))

plt.subplot(211)
plt.title('Cross Entropy Loss')
plt.plot(history.history['loss'], color='cyan', label='train')
plt.plot(history.history['val_loss'], color='magenta', label='validation')
plt.legend(loc='upper right')
    # plot accuracy
plt.subplot(212)
plt.title('Classification Accuracy')
plt.plot(history.history['accuracy'], color='cyan', label='train')
plt.plot(history.history['val_accuracy'], color='magenta', label='validation')
plt.legend(loc='upper right')
    # save plot to file
filename = sys.argv[0].split('/')[-1]
plt.savefig(filename + '_plot.png')
plt.show()

## Best model with 'rmsprop'

In [ ]:
# create data generator
datagen = ImageDataGenerator(rescale=1.0/255.0)
    
# prepare iterators
train_it = datagen.flow_from_directory(directory = train_dir,
                                        target_size=(200, 200),
                                        batch_size=64,
                                        class_mode='binary')
    
test_it = datagen.flow_from_directory(directory = val_dir,
                                      target_size=(200, 200),
                                      batch_size=64,
                                      class_mode='binary')

In [ ]:
# most
model = Sequential()

# Step 1 - Convolution
model.add(Conv2D(32, (3, 3), activation='relu', input_shape = (200, 200, 3)))
model.add(MaxPooling2D(pool_size = (3, 3)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())

model.add(Dense(64, activation='relu')) 
model.add(Dense(128, activation='relu')) 
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
batch_size = 64
history = model.fit_generator(train_it, 
                              steps_per_epoch=np.ceil(train_it.samples / batch_size),
                              validation_data=test_it,
                              validation_steps=np.ceil(test_it.samples / batch_size), 
                              epochs=20)
# evaluate model
_, acc = model.evaluate_generator(test_it, steps=len(test_it))
print('> %.3f' % (acc * 100.0))

In [ ]:
plt.style.use('bmh')
fig = plt.figure(figsize = (10,8))

plt.subplot(211)
plt.title('Cross Entropy Loss')
plt.plot(history.history['loss'], color='cyan', label='train')
plt.plot(history.history['val_loss'], color='magenta', label='validation')
plt.legend(loc='upper right')
    # plot accuracy
plt.subplot(212)
plt.title('Classification Accuracy')
plt.plot(history.history['accuracy'], color='cyan', label='train')
plt.plot(history.history['val_accuracy'], color='magenta', label='validation')
plt.legend(loc='upper right')
    # save plot to file
filename = sys.argv[0].split('/')[-1]
plt.savefig(filename + '_plot.png')
plt.show()

## Best model with 'selu' and 'rmsprop'

In [ ]:
# create data generator
datagen = ImageDataGenerator(rescale=1.0/255.0)
    
# prepare iterators
train_it = datagen.flow_from_directory(directory = train_dir,
                                        target_size=(200, 200),
                                        batch_size=64,
                                        class_mode='binary')
    
test_it = datagen.flow_from_directory(directory = val_dir,
                                      target_size=(200, 200),
                                      batch_size=64,
                                      class_mode='binary')

In [ ]:
# most
model = Sequential()

# Step 1 - Convolution
model.add(Conv2D(32, (3, 3), activation='selu', input_shape = (200, 200, 3)))
model.add(MaxPooling2D(pool_size = (3, 3)))

model.add(Conv2D(64, (3, 3), activation='selu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())

model.add(Dense(64, activation='selu')) 
model.add(Dense(128, activation='selu')) 
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
batch_size = 64
history = model.fit_generator(train_it, 
                              steps_per_epoch=np.ceil(train_it.samples / batch_size),
                              validation_data=test_it,
                              validation_steps=np.ceil(test_it.samples / batch_size), 
                              epochs=20)
# evaluate model
_, acc = model.evaluate_generator(test_it, steps=len(test_it))
print('> %.3f' % (acc * 100.0))

In [ ]:
plt.style.use('bmh')
fig = plt.figure(figsize = (10,8))

plt.subplot(211)
plt.title('Cross Entropy Loss')
plt.plot(history.history['loss'], color='cyan', label='train')
plt.plot(history.history['val_loss'], color='magenta', label='validation')
plt.legend(loc='upper right')
    # plot accuracy
plt.subplot(212)
plt.title('Classification Accuracy')
plt.plot(history.history['accuracy'], color='cyan', label='train')
plt.plot(history.history['val_accuracy'], color='magenta', label='validation')
plt.legend(loc='upper right')
    # save plot to file
filename = sys.argv[0].split('/')[-1]
plt.savefig(filename + '_plot.png')
plt.show()

# Best model with 50 epochs

## Best model with 'selu' and 'adam'

In [ ]:
# create data generator
datagen = ImageDataGenerator(rescale=1.0/255.0)
    
# prepare iterators
train_it = datagen.flow_from_directory(directory = train_dir,
                                        target_size=(200, 200),
                                        batch_size=64,
                                        class_mode='binary')
    
test_it = datagen.flow_from_directory(directory = val_dir,
                                      target_size=(200, 200),
                                      batch_size=64,
                                      class_mode='binary')

In [ ]:
model = Sequential()

# Step 1 - Convolution
model.add(Conv2D(32, (3, 3), activation='selu', input_shape = (200, 200, 3)))
model.add(MaxPooling2D(pool_size = (3, 3)))

model.add(Conv2D(64, (3, 3), activation='selu', input_shape = (200, 200, 3)))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())

model.add(Dense(64, activation='selu')) 
model.add(Dense(128, activation='selu'))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
batch_size = 64
history = model.fit_generator(train_it, 
                              steps_per_epoch=np.ceil(train_it.samples / batch_size),
                              validation_data=test_it,
                              validation_steps=np.ceil(test_it.samples / batch_size), 
                              epochs=50)
# evaluate model
_, acc = model.evaluate_generator(test_it, steps=len(test_it))
print('> %.3f' % (acc * 100.0))

In [ ]:
plt.style.use('bmh')
fig = plt.figure(figsize = (10,8))

plt.subplot(211)
plt.title('Cross Entropy Loss')
plt.plot(history.history['loss'], color='cyan', label='train')
plt.plot(history.history['val_loss'], color='magenta', label='validation')
plt.legend(loc='upper right')
    # plot accuracy
plt.subplot(212)
plt.title('Classification Accuracy')
plt.plot(history.history['accuracy'], color='cyan', label='train')
plt.plot(history.history['val_accuracy'], color='magenta', label='validation')
plt.legend(loc='upper right')
    # save plot to file
filename = sys.argv[0].split('/')[-1]
plt.savefig(filename + '_plot.png')
plt.show()

## Best model with 'relu' and 'rmsprop'

In [ ]:
# create data generator
datagen = ImageDataGenerator(rescale=1.0/255.0)
    
# prepare iterators
train_it = datagen.flow_from_directory(directory = train_dir,
                                        target_size=(200, 200),
                                        batch_size=64,
                                        class_mode='binary')
    
test_it = datagen.flow_from_directory(directory = val_dir,
                                      target_size=(200, 200),
                                      batch_size=64,
                                      class_mode='binary')

In [ ]:
model = Sequential()

# Step 1 - Convolution
model.add(Conv2D(32, (3, 3), activation='relu', input_shape = (200, 200, 3)))
model.add(MaxPooling2D(pool_size = (3, 3)))

model.add(Conv2D(64, (3, 3), activation='relu', input_shape = (200, 200, 3)))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())

model.add(Dense(64, activation='relu')) 
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
plt.style.use('bmh')
fig = plt.figure(figsize = (10,8))

plt.subplot(211)
plt.title('Cross Entropy Loss')
plt.plot(history.history['loss'], color='cyan', label='train')
plt.plot(history.history['val_loss'], color='magenta', label='validation')
plt.legend(loc='upper right')
    # plot accuracy
plt.subplot(212)
plt.title('Classification Accuracy')
plt.plot(history.history['accuracy'], color='cyan', label='train')
plt.plot(history.history['val_accuracy'], color='magenta', label='validation')
plt.legend(loc='upper right')
    # save plot to file
filename = sys.argv[0].split('/')[-1]
plt.savefig(filename + '_plot.png')
plt.show()

In [ ]:
# # save model
# model.save('model.h5')
# # 2 layers, dropout, 200x200, relu, adam, 88,4%